In [1]:
# =============================================================================
# 小売業需要予測モデル - 改善記録
# =============================================================================
# 
# 【実施した施策】
# - RandomForestRegressorのn_estimatorsを200に増加
# - より多くの決定木で予測精度を向上
# - 使用特徴量: 店舗ID, 商品ID, 年, 月, 商品カテゴリID
# 
# 【スコア】
# - ベースライン（線形回帰）: 3.9937572546850784
# - 前回の結果（rf100）: 3.07256739424164
# - 今回の結果: [提出後に記録]
# 
# 【考察】
# - [提出後に記録]
# 
# 【今後の改善案】
# - 他のハイパーパラメータの調整（max_depth, min_samples_split等）
# - 特徴量エンジニアリング（ラグ特徴量、統計特徴量等）
# - 他のモデルの試行（XGBoost, LightGBM等）
# 
# =============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import os

In [2]:
base_dir = '../data/'
sales_df = pd.read_csv(base_dir + 'sales_history.csv')
item_categories_df = pd.read_csv(base_dir + 'item_categories.csv')
category_names_df = pd.read_csv(base_dir + 'category_names.csv')
test_df = pd.read_csv(base_dir + 'test.csv')
submission_df = pd.read_csv(base_dir + 'sample_submission.csv', header=None)

In [3]:
# 日付から年と月を抽出
sales_df['年'] = sales_df['日付'].apply(lambda x: x.split('-')[0])
sales_df['月'] = sales_df['日付'].apply(lambda x: x.split('-')[1])

# 月ごとの売上個数を集計
sales_month_df = sales_df.groupby(['商品ID', '店舗ID', '年', '月'])['売上個数'].sum().reset_index()

# 商品カテゴリIDを結合
train_df = pd.merge(sales_month_df, item_categories_df, on='商品ID', how='left')

# データ型を変換
train_df['年'] = train_df['年'].astype(int)
train_df['月'] = train_df['月'].astype(int)

# test_dfにも年と月を追加し、商品カテゴリIDを結合
test_df['年'] = 2022
test_df['月'] = 12
test_df = pd.merge(test_df, item_categories_df, on='商品ID', how='left')

# 特徴量とターゲット変数を定義
feature_columns = ['店舗ID', '商品ID', '年', '月', '商品カテゴリID']
target_column = '売上個数'

# 学習データとテストデータに分割
X_train = train_df[feature_columns]
y_train = train_df[target_column]
X_test = test_df[feature_columns]

In [4]:
# モデルの定義と学習（RandomForestRegressorを使用）
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# 予測
y_pred = model.predict(X_test)

# 予測結果に負の値があれば0に変換
y_pred[y_pred < 0] = 0

print("モデルの学習が完了しました")
print(f"予測結果の形状: {y_pred.shape}")
print(f"予測値の範囲: {y_pred.min():.2f} ～ {y_pred.max():.2f}")

モデルの学習が完了しました
予測結果の形状: (3060,)
予測値の範囲: 0.68 ～ 42.08


In [5]:
# submission_df の 1 列に予測結果 (y_pred) を代入
submission_df[1] = y_pred

# 現在の日時を取得してタイムスタンプ付きファイル名を作成
import datetime
now = datetime.datetime.now()
timestamp = now.strftime("%m%d-%H%M")  # MMdd-HHmm形式

# 提出用ファイル名（改善内容とタイムスタンプ付き）
# 注意: 新しい改善を試す際は、以下の改善内容部分を変更してください
improvement_name = "rf200"  # 例: "rf200", "xgb", "features", "ensemble" など
submission_file_name = f'../submissions/20250909_Exercises3_Challenge_{improvement_name}_{timestamp}.csv'

# 提出用データフレームをCSVファイルとして出力
submission_df.to_csv(submission_file_name, index=False, header=False)

print(f"\n提出用ファイル '{submission_file_name}' が作成されました。")
print("このファイルをコンペティションサイトに提出してください。")


提出用ファイル '../submissions/20250909_Exercises3_Challenge_rf200_0909-1342.csv' が作成されました。
このファイルをコンペティションサイトに提出してください。


In [3]:
# =============================================================================
# スコア記録・分析セル
# =============================================================================
# 提出後にスコアを記録してください

# スコア記録・分析ライブラリをインポート
from score_analysis import record_score, compare_scores, generate_improvement_summary

# スコア記録（提出後に値を入力）
current_score = 3.078811806610283
model_name = "RandomForestRegressor (n_estimators=200)"
features_used = "店舗ID, 商品ID, 年, 月, 商品カテゴリID"
notes = "n_estimatorsを100から200に増加"

if current_score is not None:
    # スコアを記録・分析
    result = record_score(
        current_score=current_score,
        model_name=model_name,
        features_used=features_used,
        notes=notes
    )
else:
    print("スコアを記録してください。")
    print("例: current_score = 3.500000")
    print("その後、このセルを再実行してください。")


📊 スコア分析結果
前回のスコア: 3.072567
今回のスコア: 3.078812
改善幅: -0.006244
改善率: -0.20%
ベースラインからの改善率: 22.91%
使用モデル: RandomForestRegressor (n_estimators=200)
使用特徴量: 店舗ID, 商品ID, 年, 月, 商品カテゴリID
メモ: n_estimatorsを100から200に増加
------------------------------------------------------------
❌ スコアが悪化しました。改善が必要です。
